In [1]:
## ! pip install PyMySQL
## ! pip install ipython-sql
## ! pip install mysqlclient

In [2]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd
import pymysql

In [3]:
## Creamos una variable con el url a trabajar
r = requests.get('https://volvorepuestos.com.pe/repuestos-camiones-volvo/fm-fmx/suspension.html')

In [4]:
## Validamos que tengamos respuesta correcta con la url
r.status_code

200

In [5]:
# Creamos un variable soup para realizar las busquedas
soup = BeautifulSoup(r.content,'html.parser')

In [8]:
# Validamos que el contenido este correcto
# soup

In [9]:
## Buscamos los elementos principales en la pagina para realizar la mineria de datos

## 1.Creamos una variable dom para realizar la busquedas por ruta
dom = etree.HTML(str(soup))

## 2.Ubicamos el nombre de productos
# //ol[@class="filterproducts products list items product-items row"]//li//div[@class="product details product-item-details"]/strong[1]/a[1]/text()
nom_producto = dom.xpath('//ol[@class="filterproducts products list items product-items row"]//li//div[@class="product details product-item-details"]/strong[1]/a[1]/text()')

# Creamos una variable lista de nombre de productos con su limpieza de datos
nom_productos = [i.replace('\r','').replace('\n','').strip() for i in nom_producto]

## 3.Ubicamos la marca y codigo del producto
# marca: //ol[@class="filterproducts products list items product-items row"]//li//div[3]/div[@class="marca"]/p/span[2]/text()
# codigo: //ol[@class="filterproducts products list items product-items row"]//li//div[3]/div[@class="marca"]/p/span[3]/text()
nom_marca = dom.xpath('//ol[@class="filterproducts products list items product-items row"]//li//div[3]/div[@class="marca"]/p/span[2]/text()')
codigo = dom.xpath('//ol[@class="filterproducts products list items product-items row"]//li//div[3]/div[@class="marca"]/p/span[3]/text()')

## 4.Ubicamos el precio estimado
# //ol[@class="filterproducts products list items product-items row"]//span[@data-price-type="finalPrice"]/span[@class="price"]/text()
precio_estimado = dom.xpath('//ol[@class="filterproducts products list items product-items row"]//span[@data-price-type="finalPrice"]/span[@class="price"]/text()')
# realizamos limpieza al dato precio estimado
precio_estimado = [i.replace('$','').replace(',','').strip() for i in precio_estimado]

## 5. Ubicamos el precio base
# //ol[@class="filterproducts products list items product-items row"]//span[@data-price-type="oldPrice"]/span[@class="price"]/text()
# Para realizar la extraccion de estos precios bases realizaremos un pipe "|" con nom_productos
precio_base = dom.xpath('//ol[@class="filterproducts products list items product-items row"]//li//div[@class="product details product-item-details"]/strong[1]/a[1]/text() | //ol[@class="filterproducts products list items product-items row"]//span[@data-price-type="oldPrice"]/span[@class="price"]/text()')
# realizamos limpieza del dato precio base
precio_base = [i.replace('\r','').replace('\n','').replace(',','').strip() for i in precio_base]

In [10]:
# Validamos la lista de elementos encontrados
nom_productos

['Amortiguador Posterior',
 'Amortiguador',
 'Cojinete de ruedas',
 'Cubo (eje trasero)',
 'Kit de cojinetes',
 'Sensor',
 'Juego de cojinete de rodillos',
 'Amortiguador suspensión delantera FM12 FM13',
 'Cojinete',
 'Varilla de articulación',
 'Ballesta Delantera',
 'Cubo (eje trasero)',
 'Juego De Casquillo',
 'Servotubo',
 'Resorte Neumático',
 'Muelle Delantero',
 'Muelle Delantero',
 'Resorte de Goma',
 'Muelle de hojas',
 'Resorte Neumático',
 'Muelle Posterior',
 'Electroválvula',
 'Buje de goma',
 'Conector']

In [11]:
# Validamos que sea 24
len(nom_productos)

24

In [12]:
# Validamos Cantidad de elementos "nombre marca" sea 24
len(nom_marca)

24

In [13]:
# Validamos Cantidad de elementos "Codigos" sea 24
len(codigo)

24

In [14]:
# Validamos Cantidad de elementos "Precio" sea 24
len(precio_estimado)

24

In [15]:
# realizamos una iteracion de datos y nos quedamos solamente con los productos con precio base
# usaremos el caracter "$" para guiarnos.
data = precio_base

# Crear una nueva lista con los elementos que cumplen el criterio
nom_producto_r = [data[i] for i in range(len(data)-1) if data[i+1].startswith('$')]

# Crear una nueva lista con los elementos que cumplen el criterio
precio_antiguo_r = [data[i] for i in range(len(data)-1) if data[i].startswith('$')]

precio_base = [(nom_producto_r, precio_antiguo_r) for nom_producto_r, precio_antiguo_r in zip(nom_producto_r, precio_antiguo_r)]

In [16]:
# limpiamos por ultima vez el campo precio del caracter "$"
precio_base = [(nombre, precio.replace('$', '')) for nombre, precio in precio_base]

In [17]:
# validamos la nueva lista
precio_base

[('Cojinete de ruedas', '1052.01'),
 ('Sensor', '367.48'),
 ('Juego de cojinete de rodillos', '794.54'),
 ('Juego De Casquillo', '735.72'),
 ('Electroválvula', '609.46')]